## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("./WeatherPy_database.csv")
city_data_df.head()

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,78.53,79,75,13.80,broken clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,88,38,5.01,scattered clouds
2,2,Airai,TL,-8.9266,125.4092,58.33,87,61,1.19,broken clouds
3,3,Bengkulu,ID,-3.8004,102.2655,78.49,89,100,5.21,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,76.80,72,3,16.22,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#use a conditional filter lesson 6.5.3

good_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) &\
                                     (city_data_df['Max Temp']>= min_temp) ]
good_cities_df.head(10)

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,78.53,79,75,13.80,broken clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,88,38,5.01,scattered clouds
3,3,Bengkulu,ID,-3.8004,102.2655,78.49,89,100,5.21,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,76.80,72,3,16.22,clear sky
5,5,Hithadhoo,MV,-0.6000,73.0833,83.62,76,99,16.69,overcast clouds
6,6,Neiafu,TO,-18.6500,-173.9833,77.68,73,75,8.05,broken clouds
8,8,Roma,US,43.2128,-75.4557,75.34,23,0,10.36,clear sky
11,11,Teahupoo,PF,-17.8333,-149.2667,78.76,72,19,17.38,few clouds
13,13,Butaritari,KI,3.0707,172.7902,80.83,72,39,17.16,scattered clouds
14,14,Mitsamiouli,KM,-11.3847,43.2844,80.37,67,1,7.05,clear sky


In [13]:
# 4a. Determine if there are any empty rows.
good_cities_df.isnull()
good_cities_df.notnull()
good_cities_df.info()
## there were no empty rows located

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 717
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   city_id              230 non-null    int64  
 1   City                 230 non-null    object 
 2   Country              230 non-null    object 
 3   Lat                  230 non-null    float64
 4   Lng                  230 non-null    float64
 5   Max Temp             230 non-null    float64
 6   Humidity             230 non-null    int64  
 7   Cloudiness           230 non-null    int64  
 8   Wind Speed           230 non-null    float64
 9   Current Description  230 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 19.8+ KB


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
#changed name from clean_df to good_cities_df since I believe this new DF is supposed to come from the DF above.
hotel_df = good_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 717
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 230 non-null    object 
 1   Country              230 non-null    object 
 2   Max Temp             230 non-null    float64
 3   Current Description  230 non-null    object 
 4   Lat                  230 non-null    float64
 5   Lng                  230 non-null    float64
 6   Hotel Name           230 non-null    object 
dtypes: float64(3), object(4)
memory usage: 14.4+ KB


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row ["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       # hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
        hotels["results"][0]["name"] = hotel_df.loc[index, "Hotel Name"]
    except(IndexError):
        print("Hotel not found  . . . skipping")

Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping
Hotel not found  . . . skipping


In [18]:
# 7a. locate the rows where there is no Hotel Name.
#did not return Nan
hotel_df.notnull()
hotel_df.isnull()

#returned results
hotel_df.loc[hotel_df['Hotel Name'] == ""]

In [124]:
#7b check clean df
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,78.53,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Bengkulu,ID,78.49,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
4,Rikitea,PF,76.80,clear sky,-23.1203,-134.9692,People ThankYou
5,Hithadhoo,MV,83.62,overcast clouds,-0.6000,73.0833,Scoop Guest House


In [136]:
#check to make sure blanks are gone
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207 entries, 0 to 717
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 207 non-null    object 
 1   Country              207 non-null    object 
 2   Max Temp             207 non-null    float64
 3   Current Description  207 non-null    object 
 4   Lat                  207 non-null    float64
 5   Lng                  207 non-null    float64
 6   Hotel Name           207 non-null    object 
dtypes: float64(3), object(4)
memory usage: 12.9+ KB


In [147]:
#use .loc to check for blanks, returned no results
clean_hotel_df.loc[clean_hotel_df['Hotel Name'] == ""]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [148]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"

In [149]:
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [65]:
# 9. Using the template add city name, the country code, the weather description 
#and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [66]:
# 11a. Add a marker layer for each city to the map.
# (create a map with marker layer like lesson 6.5.4)
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 2.0)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False,
             max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))